The following is a guide to aggregating historical information for both actively managed strategies and underyling assets on the Vinter platform.

In [ ]:
# Step 1: Import dependencies and set up the environment
# version: Python 3.10.7

!pip3 install pandas
import urllib.parse
import urllib.request
import pandas as pd
from urllib import request
from json import loads
from os import environ

USERNAME = '<YOUR_USERNAME>'
PASSWORD = '<YOUR_PASSWORD>'
API_KEY = '<YOUR_API_KEY>'
TOKEN_URL='https://identity.vinter.co/authenticate/login'
ETP_URL='https://etp.vinter.co'
SINGLE_ASSET_URL='https://www.vinterapi.com/api/v3/single_assets_daily'
SYMBOL_SHORT='<SYMBOL_SHORT, EG WETI>'
SYMBOL_LONG='<SYMBOL LONG, EG vnfa-weti-15-d>'

In [ ]:
# Step 2: Use the username and password to get an access token that will be used to read PCF information.

def get_access_token():
  # url = TOKEN_URL
  formdata = {
      "username": USERNAME,
      "password": PASSWORD
  }
  headers = {
      "Content-Type": "application/x-www-form-urlencoded"
  }
  data = urllib.parse.urlencode(formdata).encode()
  req = urllib.request.Request(TOKEN_URL, headers=headers, data=data)
  with urllib.request.urlopen(req) as response:
    intermediate_json_value = response.read().decode()
    decoded = loads(intermediate_json_value)

    token = decoded['access_token']
    return token

token = get_access_token()

In [ ]:
# Step 3: Use the token to get the PCF information
# import urllib.request

def get_pcf_info():
  url = f"{ETP_URL}/api/v1/nav/ticker/{SYMBOL_SHORT}"
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
  }
  req = request.Request(url, headers=headers)
  with request.urlopen(req) as response:
    intermediate_json_value = loads(response.read().decode())
    return intermediate_json_value

pcf_info = get_pcf_info()
print(pcf_info)
print("Index Info:")
print("Full Name:" + pcf_info['full_name'])
print("Outstanding shares:" + str(pcf_info['outstanding_shares']))
print("ID" + str(pcf_info['id']))

def index_data_to_constituents(data):
  pcf = data["pcf"]
  # map pcf to [ticker, amount]

  mapped =  map(lambda x: [x['ticker'], x['amount']], pcf)
  return list(mapped)

constituents = index_data_to_constituents(pcf_info)
print("Listing constituents:")
for c in constituents:
  print(c[0] + "//" + str(c[1]))


In [ ]:
# Finally, let's gather historical data for the indexes

headers = {
  "authorization": API_KEY
}

for (ticker, amount) in constituents:
  ticker_long = f"{ticker.lower()}-usd-p-d" # for accessing the reference rate relative to USD
  if (ticker == "USD"):
    # for this computation, we will skip USD, since the rate of USD to USD is always 1
    continue
  if (ticker == "EUR"):
    ticker_long = "usd-eur-p-d"
  url = f"{SINGLE_ASSET_URL}?symbol={ticker_long}&limit=200"
  print(url)
  req = request.Request(url, headers=headers)
  try:
    with request.urlopen(req) as response:
      intermediate_json_value = loads(response.read().decode())
      df = pd.DataFrame(intermediate_json_value['data'])
      print(f"Data for {ticker}")
      print(df.tail())
      # print(intermediate_json_value)
  except Exception as e:
    print(e)
    print("Error getting data for " + ticker)
    continue